# Survey features checklist
This notebook provides a checklist of key features of EDSL for use in reviewing the logic and contents of a survey and ensuring that it will run as you intend it to.

## Support
Please reach out to us with any questions or issues! 
Send an email to **info@expectedparrot.com** or post a message at [Discord]().

## Understanding how EDSL works
At its core, EDSL is built on the concept of a [`Question`](https://docs.expectedparrot.com/en/latest/questions.html) being answered by an AI [`Agent`](https://docs.expectedparrot.com/en/latest/agents.html), which uses a large language [`Model`](https://docs.expectedparrot.com/en/latest/language_models.html) to generate a response, returning it as a formatted [`Result`](https://docs.expectedparrot.com/en/latest/results.html). 
A typical workflow consists of the following steps:

1. Construct questions of different types (multiple choice, free text, etc.) and combine them in a survey. Learn about [question types](https://docs.expectedparrot.com/en/latest/questions.html).
2. *Optional:* Add data or content to your questions (from CSVs, PDFs, docs, images, etc.). Learn about [scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html).
3. *Optional:* Add rules or logic to specify how questions should be presented (e.g., skip/stop rules, or including context of other questions). Learn about [survey rules](https://docs.expectedparrot.com/en/latest/surveys.html).
4. *Optional:* Design or import personas for AI agents to answer the questions. Learn about [agents](https://docs.expectedparrot.com/en/latest/agents.html).
5. Select language models to generate responses. EDSL works with hundreds of popular [models](https://docs.expectedparrot.com/en/latest/language_models.html) to choose from.
6. Run the survey and get a formatted dataset of results. Use built-in methods for analyzing them.
7. *Optional:* Store and share your work at the [Coop](https://www.expectedparrot.com/content/explore): a platform for AI research that is fully integrated with EDSL. Learn more.

## Key features

* **Skip/stop rules:** Is logic for skipping questions correctly expressed in the rules?
* **Question memory:** Does any `question_text` depend on information in or responses to other questions in the survey, either impliedly or expressly?

## Common errors


In reviewing your survey, there may be questions that require information from other questions in the survey, either in whole or in part.

**Incomplete question options** 
* Use `QuestionYesNo` when you only want to allow "Yes" or "No" responses.
* Use `QuestionMultipleChoice` when you want a model to select one option only.
* Use `QuestionCheckBox` when you want to allow a model to make multiple (or no) selections, or specify a number of selections.

**Wrong question type**

**Missing context**
* The model does not have enough information to answer the question. Context of another question is needed, e.g., the response, or any content that has been added to a prior question for the model to evaluate. Learn about [piping]() questions and adding [question memory]().
* A question text says multiple options may be selected, but the question type is multiple choice.
* The question options c



## Tools for auto-importing survey data
EDSL also provides features for importing data to use in surveys:

### Conjure
The `Conjure` module allows you to impoort a file of survey data (SPSS, CSV) and automatically generate EDSL objects based on it:

* A `Survey` consisting of the original survey questions.
* An `AgentList` of agents representing the original respondents, with traits for their original responses.
* A `Results` dataset of the original responses that you can analyze with EDSL's built-in methods for analyzing any survey responses.

[Learn more]().


### Tools for generating question scenarios
EDSL provides many methods for automatically formatting data or content to be used with a survey.
[Learn more]() about creating and using `Scenario` and `ScenarioList` objects for texts, docs, PDFs, images, CSVs, tables and other data types that you want to add to survey questions or store with survey results as metadata. 

## Skip/stop rules

## Question memory
`Question` objects are combined in a `Survey` in order to administer them to a model (or multiple models) at once.
By default, questions are administered asynchronously, meaning that the presentation of one question does not include any context of other questions in the survey, unless you specify otherwise. 
This default functionality saves time and reduces token costs.
It also allows you to fine tune and readily compare responses with different contexts.



In [ ]:
from edsl import QuestionYesNo, QuestionFreeText, Survey

q_own_car = QuestionYesNo(
    question_name = "own_car",
    question_text = "Do you currently own a car?"
)

q_color = QuestionFreeText(
    question_name = "color",
    question_text = "What color is your car?" # We need to add a rule to skip this question if the prior answer is "No"
)

q_commute = QuestionMultipleChoice(
    question_name = "commute_car",
    question_text = "How often do you commute to work by car?" # We may want to add the context of the answer to the first question for coherence
)

survey = Survey([q_own_car, q_color])

### Skip rule

In [ ]:
skip_survey = survey.add_skip_rule("color", "own_car == 'No'")

### Stop rule

In [ ]:
stop_survey = survey.add_stop_rule("commute_car", "own_car")

### Targeted question memory

In [ ]:
targeted_memory_survey = survey.add_targeted_memory("color", "own_car == 'No'")

### Full question memory

In [ ]:
full_memory_survey = survey.set_full_memory_mode()